In [4]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [5]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [6]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [7]:
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import _fetch_batch, fetch_data, fetch_spot_data

In [8]:
from expiries import dict_expiries

In [9]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [10]:
bnf_1min = pd.read_csv("../data/nifty_min.csv")

In [11]:
bnf_1min.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_1min.tail()

,index,datetime,o,h,l,c,v
743635,nifty,2025-02-28 15:25:00,22110.4500,22119.0500,22109.2500,22118.8000,0
743636,nifty,2025-02-28 15:26:00,22118.6000,22123.8500,22113.2000,22114.7000,0
743637,nifty,2025-02-28 15:27:00,22114.0000,22120.1500,22108.4500,22114.1500,0
743638,nifty,2025-02-28 15:28:00,22114.1500,22121.2500,22106.1000,22113.8500,0
743639,nifty,2025-02-28 15:29:00,22112.5500,22127.4000,22109.3500,22127.4000,0


In [12]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"]).dt.tz_localize(None)
# bnf_1min = bnf_1min[((bnf_1min['datetime'].dt.year == 2020) & (bnf_1min['datetime'].dt.month == 4))]
bnf_1min = bnf_1min[
    (bnf_1min["datetime"].dt.year >= 2019) & (bnf_1min["datetime"].dt.year <= 2025)
]

In [13]:
dict_expiries

{'nifty': [datetime.datetime(2017, 1, 25, 0, 0),
  datetime.datetime(2017, 2, 23, 0, 0),
  datetime.datetime(2017, 3, 30, 0, 0),
  datetime.datetime(2017, 4, 27, 0, 0),
  datetime.datetime(2017, 5, 25, 0, 0),
  datetime.datetime(2017, 6, 29, 0, 0),
  datetime.datetime(2017, 7, 27, 0, 0),
  datetime.datetime(2017, 8, 31, 0, 0),
  datetime.datetime(2017, 9, 28, 0, 0),
  datetime.datetime(2017, 10, 26, 0, 0),
  datetime.datetime(2017, 11, 30, 0, 0),
  datetime.datetime(2017, 12, 28, 0, 0),
  datetime.datetime(2018, 1, 25, 0, 0),
  datetime.datetime(2018, 2, 22, 0, 0),
  datetime.datetime(2018, 3, 28, 0, 0),
  datetime.datetime(2018, 4, 26, 0, 0),
  datetime.datetime(2018, 5, 31, 0, 0),
  datetime.datetime(2018, 6, 28, 0, 0),
  datetime.datetime(2018, 7, 26, 0, 0),
  datetime.datetime(2018, 8, 30, 0, 0),
  datetime.datetime(2018, 9, 27, 0, 0),
  datetime.datetime(2018, 10, 25, 0, 0),
  datetime.datetime(2018, 11, 29, 0, 0),
  datetime.datetime(2018, 12, 27, 0, 0),
  datetime.datetime(2019,

In [14]:
from datetime import date
from bisect import bisect_right

def get_next_expiry(input_date, index_symbol):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_right(expiry_dates, input_date)
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [15]:
index_ = 'nifty'

if index_ == 'nifty':
    LOT_SIZE_ = 75
    STRIKE_SPREAD_ = 50
    INDEX_LEVERAGE_ = 8

In [44]:
async def execute_daily_strangle_strategy(df, x_percent, tf, offset):
    df['date'] = df['datetime'].dt.date
    df['time'] = df['datetime'].dt.time
    
    # Daily levels (excluding today)
    daily_levels = df.groupby('date').agg({'h': 'max', 'l': 'min'})
    daily_levels['range'] = daily_levels['h'] - daily_levels['l']
    daily_levels['prev_high'] = daily_levels['h'].shift(1)
    daily_levels['prev_low'] = daily_levels['l'].shift(1)
    daily_levels['prev_range'] = daily_levels['range'].shift(1)
    tradebook = []

    for i in range(1, len(daily_levels)):
        day = daily_levels.index[i]
        day_df = df[df['date'] == day].copy()
        current_open = day_df.iloc[0]['o']
        print(day)
        # print(day_df.to_string())
        # break
        if day_df.empty:
            continue
        
        # Entry levels based on previous day
        prev_high = daily_levels.iloc[i]['prev_high']
        prev_low = daily_levels.iloc[i]['prev_low']
        prev_range = daily_levels.iloc[i]['prev_range']
        
        # high_level = current_open + (prev_range * x_percent)
        # low_level = current_open - (prev_range * x_percent)

        high_level = prev_high + (prev_range * x_percent)
        low_level = prev_low - (prev_range * x_percent)
        
        # print(prev_high, prev_low, prev_range, high_level, low_level)

        high_level_strike = np.round(current_open / STRIKE_SPREAD_) * STRIKE_SPREAD_
        low_level_strike = np.round(current_open / STRIKE_SPREAD_) * STRIKE_SPREAD_

        expiry = get_next_expiry(day, index_)
        dte = (expiry - day).days

        ce_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(high_level_strike),
            asset_class='C',
            start_date=day,
            start_time=dt.time(9, 16),
            end_date=day,
            end_time=dt.time(15, 30),
        )
        if not isinstance(ce_df, str) and ce_df is not None:
            ce_df = resample(ce_df, tf, offset)
            ce_df_pandas = ce_df.to_pandas()
            ce_entry_price = ce_df_pandas.iloc[0]['o']
        else:
            ce_entry_price = float('nan')

        pe_df = await fetch_data(
            index=index_,
            expiry=expiry,
            strike=int(low_level_strike),
            asset_class='P',
            start_date=day,
            start_time=dt.time(9, 16),
            end_date=day,
            end_time=dt.time(15, 30),
        )
        if not isinstance(pe_df, str) and pe_df is not None:
            pe_df = resample(pe_df, tf, offset)
            pe_df_pandas = pe_df.to_pandas()
            pe_entry_price = pe_df_pandas.iloc[0]['o']
        else:
            pe_entry_price = float('nan')

        # print(ce_entry_price , pe_entry_price)
         
        tradebook.append({
            'date': day,
            'entry_time': dt.datetime.combine(day, dt.time(9, 16)),
            'day open': day_df['o'].iloc[0],
            'yesterday high': daily_levels['prev_high'].iloc[i],
            'yesterday low': daily_levels['prev_low'].iloc[i],
            'yesterday range': daily_levels['prev_range'].iloc[i],
            'multiplier%': x_percent*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(high_level_strike),
            'type': 'CE',
            'expiry': expiry,
            "tag": "STRANGLE",
            'entry price': ce_entry_price,
        })
        tradebook.append({
            'date': day,
            'entry_time': dt.datetime.combine(day, dt.time(9, 16)),
            'day open': day_df['o'].iloc[0],
            'yesterday high': daily_levels['prev_high'].iloc[i],
            'yesterday low': daily_levels['prev_low'].iloc[i],
            'yesterday range': daily_levels['prev_range'].iloc[i],
            'multiplier%': x_percent*100,
            'high level': high_level, 
            'low level': low_level,
            'strike': int(low_level_strike),
            'type': 'PE',
            'expiry': expiry,
            "tag": "STRANGLE",
            'entry price': pe_entry_price,
        })

        ce_trade_active = True
        pe_trade_active = True
        
        for idx, row in day_df.iterrows():
            
            curr_datetime = row['datetime']
            
            if ce_trade_active and row['c'] >= high_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'CE' and 
                        trade['date'] == day and 
                        'exit_time' not in trade
                    ):
                        df_row = ce_df_pandas[ce_df_pandas['datetime'] <= curr_datetime]
                        ce_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = ce_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('CE Exit', ce_exit_price)
                        break
                ce_trade_active = False  # Mark CE trade as inactive

            if pe_trade_active and row['c'] <= low_level:
                for trade in tradebook:
                    if (
                        trade['type'] == 'PE' and 
                        trade['date'] == day and 
                        'exit_time' not in trade
                    ):
                        df_row = pe_df_pandas[pe_df_pandas['datetime'] <= curr_datetime]
                        pe_exit_price = df_row['c'].iloc[-1] if len(df_row) != 0 else float('nan')
                        trade['exit price'] = pe_exit_price
                        trade['exit_time'] = curr_datetime
                        # print('PE Exit', pe_exit_price)
                        break
                pe_trade_active = False  # Mark PE trade as inactive

            # Re-enter if closed back inside range
            if not ce_trade_active and row['c'] <= high_level:
                df_row = ce_df_pandas[ce_df_pandas['datetime'] >= curr_datetime]
                ce_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('CE Entry', ce_entry_price)
                tradebook.append(
                    {
                        'date': day,
                        'entry_time': curr_datetime,
                        'day open': day_df['o'].iloc[0],
                        'yesterday high': daily_levels['prev_high'].iloc[i],
                        'yesterday low': daily_levels['prev_low'].iloc[i],
                        'yesterday range': daily_levels['prev_range'].iloc[i],
                        'multiplier%': x_percent*100,
                        'high level': high_level, 
                        'low level': low_level,
                        'strike': int(high_level_strike),
                        'type': 'CE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        'entry price': ce_entry_price,
                    }
                )
                ce_trade_active = True  # Reactivate CE trade
        
            if not pe_trade_active and row['c'] >= low_level:
                df_row = pe_df_pandas[pe_df_pandas['datetime'] >= curr_datetime]
                pe_entry_price = df_row['c'].iloc[0] if len(df_row) != 0 else float('nan')
                # print('PE Entry', pe_entry_price)
                tradebook.append(
                    {
                        'date': day,
                        'entry_time': curr_datetime,
                        'day open': day_df['o'].iloc[0],
                        'yesterday high': daily_levels['prev_high'].iloc[i],
                        'yesterday low': daily_levels['prev_low'].iloc[i],
                        'yesterday range': daily_levels['prev_range'].iloc[i],
                        'multiplier%': x_percent*100,
                        'high level': high_level, 
                        'low level': low_level,
                        'strike': int(high_level_strike),
                        'type': 'PE',
                        'expiry': expiry,
                        "tag": "RE-ENTRY",
                        'entry price': pe_entry_price,
                    }
                )
                pe_trade_active = True  # Reactivate CE trade

            if curr_datetime.time() >= dt.time(15, 14):
                for trade in tradebook:
                    if (
                        trade['date'] == day and 
                        'exit_time' not in trade
                    ):
                        trade['exit_time'] = curr_datetime

                        if trade['type'] == 'CE':
                            trade['exit price'] = ce_df_pandas['c'].iloc[-1]
                            # print('CE Exit', ce_df_pandas['c'].iloc[-1])
                        else:
                            trade['exit price'] = pe_df_pandas['c'].iloc[-1]
                            # print('PE Exit', pe_df_pandas['c'].iloc[-1])

    tb = pd.DataFrame(tradebook)
    tb['entry_time'] = pd.to_datetime(tb['entry_time'])
    tb['expiry'] = pd.to_datetime(tb['expiry'])
    tb['dte'] = (tb['expiry'].dt.date - tb['entry_time'].dt.date).apply(lambda x: x.days)
    tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
    tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
    tb['portfolio'] = 1_00_00_000
    tb['index leverage'] = np.where(tb['type'] == 'PE', 8, 8)
    tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike'] 
    tb['PnL'] = tb['qty'] * tb['final points']
    tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
    tb['Trade Year'] = tb['entry_time'].dt.year

    return pd.DataFrame(tb)


In [45]:
tf = '20m'
offset = '15m'
bnf_1h = resample(pl.DataFrame(bnf_1min), tf, offset)
tb = await execute_daily_strangle_strategy(bnf_1h.to_pandas(), 0.9, tf, offset)

2019-01-02
2019-01-03
2019-01-04
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-18
2019-03-19
2019-03-20
2019-03-22
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-04-01
2019-04-02
2019-04-03
2019-04-04
2019-04-05
2019-04-08
2019-04-09
2019-04-10
2019-04-11
2019-04-12
2019-04-15
2019-04-16
2019-04-18
2019-04-22
2019-04-23
2019-04-24
2019-04-25
2019-04-26
2019-04-30
2019-05-02
2019-05-03
2019-05-06
2019-05-07
2019-05-08
2019-05-09
2019-05-10
2019-05-13
2019-05-14
2019-05-15
2019-05-16

In [46]:
tb.tail()

,date,entry_time,day open,yesterday high,yesterday low,yesterday range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit_time,exit price,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
3296,2025-02-25,2025-02-25 09:16:00,22516.4500,22668.0500,22518.8000,149.2500,90.0000,22802.3750,22384.4750,22500,PE,2025-02-27,STRANGLE,60.2000,2025-02-25 15:15:00,34.1000,2,0.9430,25.1570,10000000,8,3555.5556,89447.1111,0.8945,2025
3297,2025-02-27,2025-02-27 09:16:00,22568.9500,22625.3000,22513.9000,111.4000,90.0000,22725.5600,22413.6400,22550,CE,2025-03-06,STRANGLE,204.0500,2025-02-27 15:15:00,161.2500,7,3.6530,39.1470,10000000,8,3547.6718,138880.7095,1.3888,2025
3298,2025-02-27,2025-02-27 09:16:00,22568.9500,22625.3000,22513.9000,111.4000,90.0000,22725.5600,22413.6400,22550,PE,2025-03-06,STRANGLE,124.7500,2025-02-27 15:15:00,130.0000,7,2.5475,-7.7975,10000000,8,3547.6718,-27662.9712,-0.2766,2025
3299,2025-02-28,2025-02-28 09:16:00,22433.4000,22613.3000,22508.4000,104.9000,90.0000,22707.7100,22413.9900,22450,CE,2025-03-06,STRANGLE,132.8000,2025-02-28 15:15:00,42.2500,6,1.7505,88.7995,10000000,8,3563.4744,316434.7439,3.1643,2025
3300,2025-02-28,2025-02-28 09:16:00,22433.4000,22613.3000,22508.4000,104.9000,90.0000,22707.7100,22413.9900,22450,PE,2025-03-06,STRANGLE,170.8000,2025-02-28 09:15:00,205.4500,6,3.7625,-38.4125,10000000,8,3563.4744,-136881.9599,-1.3688,2025


In [47]:
# tb['slippage'] = 0.01 * (tb['entry price'] + tb['exit price'])
# tb['final points'] = tb['entry price'] - tb['exit price'] - tb['slippage']
# tb['portfolio'] = 1_00_00_000
# tb['index leverage'] = 8
# tb['qty'] = tb['portfolio'] * tb['index leverage'] / tb['strike']
# tb['PnL'] = tb['qty'] * tb['final points']
# tb['ROI%'] = tb['PnL'] * 100 / tb['portfolio']
# tb['Trade Year'] = tb['week_start'].dt.year

In [48]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [49]:
tb[tb['Trade Year'] == 2020]

,date,entry_time,day open,yesterday high,yesterday low,yesterday range,multiplier%,high level,low level,strike,type,expiry,tag,entry price,exit_time,exit price,dte,slippage,final points,portfolio,index leverage,qty,PnL,ROI%,Trade Year
516,2020-01-01,2020-01-01 09:16:00,12202.1500,12247.1000,12152.2500,94.8500,90.0000,12332.4650,12066.8850,12200,CE,2020-01-02,STRANGLE,40.2500,2020-01-01 15:15:00,24.0000,1,0.6425,15.6075,10000000,8,6557.3770,102344.2623,1.0234,2020
517,2020-01-01,2020-01-01 09:16:00,12202.1500,12247.1000,12152.2500,94.8500,90.0000,12332.4650,12066.8850,12200,PE,2020-01-02,STRANGLE,29.9000,2020-01-01 15:15:00,25.8500,1,0.5575,3.4925,10000000,8,6557.3770,22901.6393,0.2290,2020
518,2020-01-02,2020-01-02 09:16:00,12198.5500,12222.0500,12165.4000,56.6500,90.0000,12273.0350,12114.4150,12200,CE,2020-01-09,STRANGLE,78.2500,2020-01-02 14:55:00,125.8000,7,2.0405,-49.5905,10000000,8,6557.3770,-325183.6066,-3.2518,2020
519,2020-01-02,2020-01-02 09:16:00,12198.5500,12222.0500,12165.4000,56.6500,90.0000,12273.0350,12114.4150,12200,PE,2020-01-09,STRANGLE,48.0000,2020-01-02 15:15:00,23.7000,7,0.7170,23.5830,10000000,8,6557.3770,154642.6230,1.5464,2020
520,2020-01-03,2020-01-03 09:16:00,12261.1000,12289.7500,12198.2000,91.5500,90.0000,12372.1450,12115.8050,12250,CE,2020-01-09,STRANGLE,61.6000,2020-01-03 15:15:00,48.2000,6,1.0980,12.3020,10000000,8,6530.6122,80339.5918,0.8034,2020
521,2020-01-03,2020-01-03 09:16:00,12261.1000,12289.7500,12198.2000,91.5500,90.0000,12372.1450,12115.8050,12250,PE,2020-01-09,STRANGLE,65.8500,2020-01-03 15:15:00,73.8000,6,1.3965,-9.3465,10000000,8,6530.6122,-61038.3673,-0.6104,2020
522,2020-01-06,2020-01-06 09:16:00,12170.6000,12265.5000,12192.4000,73.1000,90.0000,12331.2900,12126.6100,12150,CE,2020-01-09,STRANGLE,62.9000,2020-01-06 15:15:00,15.2500,3,0.7815,46.8685,10000000,8,6584.3621,308599.1770,3.0860,2020
523,2020-01-06,2020-01-06 09:16:00,12170.6000,12265.5000,12192.4000,73.1000,90.0000,12331.2900,12126.6100,12150,PE,2020-01-09,STRANGLE,66.0500,2020-01-06 09:15:00,109.4500,3,1.7550,-45.1550,10000000,8,6584.3621,-297316.8724,-2.9732,2020
524,2020-01-07,2020-01-07 09:16:00,12079.1000,12175.4500,11974.3500,201.1000,90.0000,12356.4400,11793.3600,12100,CE,2020-01-09,STRANGLE,60.6500,2020-01-07 15:15:00,37.0000,2,0.9765,22.6735,10000000,8,6611.5702,149907.4380,1.4991,2020
525,2020-01-07,2020-01-07 09:16:00,12079.1000,12175.4500,11974.3500,201.1000,90.0000,12356.4400,11793.3600,12100,PE,2020-01-09,STRANGLE,52.3500,2020-01-07 15:15:00,66.0000,2,1.1835,-14.8335,10000000,8,6611.5702,-98072.7273,-0.9807,2020


In [50]:
stats = generate_stats(tb, 'RBDS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,48.4076,516,54.0698,1.9270,-2.3297,-22.2101,2.1795,RBDS
2020,-15.2515,550,54.7273,2.9849,-3.9215,-155.8537,-0.0979,RBDS
2021,48.4486,536,55.0373,2.1169,-2.5488,-43.4095,1.1161,RBDS
2022,3.7675,548,51.2774,2.3065,-2.5671,-33.2219,0.1134,RBDS
2023,27.9176,527,53.3207,1.4210,-1.6433,-20.6032,1.3550,RBDS
2024,31.6583,532,55.4511,1.8391,-2.2910,-37.5351,0.8434,RBDS
2025,25.1320,92,58.6957,1.9933,-2.1712,-10.3619,2.4254,RBDS
Overall,170.0800,3301,54.1048,2.1043,-2.5502,-155.8537,1.0913,RBDS


In [43]:
stats = generate_stats(tb, 'RBDS')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,53.7076,516,62.4031,0.7378,-1.1010,-11.1103,4.8340,RBDS
2020,43.7448,550,60.7273,1.1316,-1.6965,-27.7682,1.5754,RBDS
2021,41.4770,536,63.9925,0.7802,-1.2703,-14.6327,2.8345,RBDS
2022,27.2590,548,58.7591,0.9260,-1.2901,-23.2785,1.1710,RBDS
2023,14.3984,527,62.8083,0.4699,-0.8020,-11.4144,1.2614,RBDS
2024,18.2332,532,65.9774,0.6475,-1.2593,-21.6995,0.8403,RBDS
2025,7.9641,92,64.1304,0.8037,-1.1956,-6.5072,1.2239,RBDS
Overall,206.7842,3301,62.4659,0.7815,-1.2464,-27.7682,7.4468,RBDS


In [26]:
tb_strangle = tb[tb['tag'] == 'STRANGLE']

In [27]:
stats = generate_stats(tb_strangle, 'RBW w Strangle')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,42.9798,488,63.7295,0.7050,-1.1520,-11.0408,3.8928,RBW w Strangle
2020,26.6038,504,63.4921,1.0480,-1.8713,-32.2235,0.8256,RBW w Strangle
2021,32.6544,496,66.5323,0.7401,-1.3920,-14.3165,2.2809,RBW w Strangle
2022,30.5750,496,61.4919,0.9001,-1.3941,-20.4784,1.4930,RBW w Strangle
2023,10.0393,492,64.8374,0.4510,-0.8747,-11.5779,0.8671,RBW w Strangle
2024,21.6600,498,68.4739,0.6315,-1.3640,-15.9947,1.3542,RBW w Strangle
2025,8.8660,86,66.2791,0.8114,-1.2890,-6.5251,1.3588,RBW w Strangle
Overall,173.3782,3060,64.8039,0.7458,-1.3472,-33.6489,5.1526,RBW w Strangle


In [92]:
# SIMULATION
multiplier_range = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.66, 0.75, 0.9, 1]
tf = ['1m', '3m', '5m', '10m', '15m', '20m']

for i in multiplier_range:
    for j in tf:
        variation = f'Multiplier {i}, TF {j}'
        print(variation)
        
        if j == '10m':
            offset = '5m'
        elif j >= '20m':
            offset = '15m'
        else:
            offset = '0m'
            
        bnf_1h = resample(pl.DataFrame(bnf_1min), j, offset)
        tb = await execute_daily_strangle_strategy(bnf_1h.to_pandas(), i, j, offset)

        if len(tb) > 0:
            stats = generate_stats(tb, variation)
            for x, y in stats.items():
                z = pd.DataFrame(y)
                print(z.to_string())


Multiplier 0.05, TF 1m
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio               Variation
2019     -31.0961         1292  25.0000                1.2491             -0.5024     -39.5125      -0.7870  Multiplier 0.05, TF 1m
2020     -48.4885         1319  25.3222                1.8641             -0.7776     -62.6795      -0.7736  Multiplier 0.05, TF 1m
2021       3.2053         1351  24.7964                1.2851             -0.4780     -31.6443       0.1013  Multiplier 0.05, TF 1m
2022     -48.0794         1267  23.9148                1.4696             -0.6069     -53.6434      -0.8963  Multiplier 0.05, TF 1m
2023     -37.3048         1270  23.4646                0.8891             -0.3535     -47.1357      -0.7914  Multiplier 0.05, TF 1m
2024     -20.9574         1351  26.2028                1.1266             -0.4820     -46.0459      -0.4551  Multiplier 0.05, TF 1m
2025       5.1416          222  28.3784              